In [8]:
!pip install -q langchain langchain_core langchain_community sentence_transformers faiss-cpu langchain_groq langchain_huggingface pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.6 MB/s eta 0:00:00


In [12]:
GROQ_API_TOKEN = 'Your GROQ API TOKEN'

In [3]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA


Steps:
1. Load PDF
2. Splitting Character from Text as Chunks
3. Create Embeddings
4. Store Embeddings as vector store
5. Initialize the LLM model
6. Creating retrieval for QA
7. Taking USer Question

Then the answer is returned as reponse

PDF file is provided Here

In [6]:
pdf_path = '/content/Knowledge_Base.pdf'

The PDF is first loaded

In [9]:
loader = PyPDFLoader(pdf_path)
document = loader.load()

The characters are splitted from text at size of 1000(chunks), to maintain the context the overlap of 200 charcter is set

In [10]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(document)

Embeddings are created using a light-weight embeddings model: **sentence-transformers/all-MiniLM-L6-v2**

In [11]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The embeddings are stored as vector store

In [18]:
vectorstore = FAISS.from_documents(docs, embeddings)

The GROQ LLM is initialized

In [16]:
GROQ_LLM = ChatGroq(
    api_key = GROQ_API_TOKEN,
    model = 'gemma2-9b-it'
)

Retrieval is created with GROQ LLM and FAISS vectorstore

In [19]:
chain = RetrievalQA.from_chain_type(
    llm = GROQ_LLM,
    chain_type = 'stuff',
    retriever = vectorstore.as_retriever(),
)

User's query is taken

In [20]:
user_input = input("Ask your Question: ")

Ask your Question: What is Canker


In [23]:
response = chain.invoke(user_input)
print(response['result'])

Canker is a bacterial disease that appears as brownish-yellow spots on the leaves, stems, and fruits of citrus trees.  

